In [118]:
import pandas as pd
import numpy as np

In [134]:
df1 = pd.read_csv('PositiveAreas.csv')
df2 = pd.read_csv('EPA mixes.csv')

In [135]:
df1['Average Mz'] = df1['Average Mz'] - 1.007825035

In [139]:
df1['Mass_round'] = np.round(df1['Average Mz'],2)
df2['Mass_round'] = np.round(df2['Monoisotopic mass'],2)

In [140]:
df = pd.merge(df1, df2, on = 'Mass_round', how = 'inner')

In [142]:
df.head()

,Alignment ID,Average Rt(min),Average Mz,Metabolite name,Adduct type,Post curation result,Fill %,MS/MS assigned,Reference RT,Reference m/z,...,QC-50ppb-pos,Average,Stdev,Mass_round,EPA names,Solvent,Chemicals,Molecular Formula,Molecular weight (g/mol),Monoisotopic mass
0,540,2.529,127.942785,Unknown,[M+H]+,NaN,0.2,False,NaN,NaN,...,535529,1.642177e+06,9.070273e+05,127.94,EPA 552.2 haloacetic acids mix,methyl tert-butyl ether,Dichloroacetic acid,C2H2Cl2O2,128.94,127.943185
1,541,2.420,127.942815,Unknown,[M+H]+,NaN,0.4,False,NaN,NaN,...,470660,1.631492e+06,9.704790e+05,127.94,EPA 552.2 haloacetic acids mix,methyl tert-butyl ether,Dichloroacetic acid,C2H2Cl2O2,128.94,127.943185
2,1104,1.119,161.960745,Unknown,[M+H]+,NaN,0.4,False,NaN,NaN,...,11854822,1.486101e+07,6.690301e+06,161.96,EPA 8270 phenols mix,isopropanol,"2,4-Dichlorophenol",C6H4Cl2O,163,161.963920
3,1104,1.119,161.960745,Unknown,[M+H]+,NaN,0.4,False,NaN,NaN,...,11854822,1.486101e+07,6.690301e+06,161.96,EPA 8270 phenols mix,isopropanol,"2,6-Dichlorophenol",C6H4Cl2O,163,161.963920
4,1105,3.370,161.960745,Unknown,[M+H]+,NaN,0.2,False,NaN,NaN,...,8473748,2.757915e+07,2.120713e+07,161.96,EPA 8270 phenols mix,isopropanol,"2,4-Dichlorophenol",C6H4Cl2O,163,161.963920


In [146]:
df['mass_diff'] = ((np.absolute(df['Average Mz'] - df['Monoisotopic mass']))/df['Monoisotopic mass'])*10**6
df['mass_flag'] = np.where(df['mass_diff'] > 5, 1, 0)

In [147]:
df.columns.values

array(['Alignment ID', 'Average Rt(min)', 'Average Mz', 'Metabolite name',
       'Adduct type', 'Post curation result', 'Fill %', 'MS/MS assigned',
       'Reference RT', 'Reference m/z', 'Formula', 'Ontology', 'INCHIKEY',
       'SMILES', 'Annotation tag (VS1.0)', 'RT matched', 'm/z matched',
       'MS/MS matched', 'Comment', 'Manually modified for quantification',
       'Manually modified for annotation', 'Isotope tracking parent ID',
       'Isotope tracking weight number', 'Total score', 'RT similarity',
       'Dot product', 'Reverse dot product', 'Fragment presence %',
       'S/N average', 'Spectrum reference file name',
       'MS1 isotopic spectrum', 'MS/MS spectrum', 'QC-1000ppb-pos',
       'QC-100ppb-pos', 'QC-200ppb-pos', 'QC-500ppb-pos', 'QC-50ppb-pos',
       'Average', 'Stdev', 'Mass_round', 'EPA names', 'Solvent ',
       'Chemicals', 'Molecular Formula', 'Molecular weight (g/mol)',
       'Monoisotopic mass', 'mass_diff', 'mass_flag'], dtype=object)

In [148]:
loc1 = df.loc[:, 'Alignment ID':'MS/MS spectrum']
loc2 = df.loc[:, 'Mass_round':'mass_flag']

In [149]:
dft = df[['QC-1000ppb-pos', 'QC-100ppb-pos', 'QC-200ppb-pos', 'QC-500ppb-pos', 'QC-50ppb-pos']]
# this is for log10 regression
dft = dft.replace(0, 0.01)
dft = np.log10(dft)

In [150]:
axisvalues_ = [1000, 500, 200, 100, 50]

In [151]:
from scipy import stats
from scipy.stats import linregress

dft = dft.astype(float)
def calc_slope(row):
    a = scipy.stats.linregress(axisvalues_, row)
    return pd.Series(a._asdict())

In [152]:
linr_res = dft.apply(calc_slope,axis=1)

In [153]:
linr_res

,slope,intercept,rvalue,pvalue,stderr,intercept_stderr
0,0.000351,6.019463,0.484411,0.408279,0.000366,0.186804
1,0.000208,6.058208,0.263103,0.668913,0.000441,0.224914
2,0.000100,7.104428,0.220532,0.721502,0.000254,0.129777
3,0.000100,7.104428,0.220532,0.721502,0.000254,0.129777
4,-0.000044,7.346786,-0.049319,0.937231,0.000514,0.262402
5,-0.000044,7.346786,-0.049319,0.937231,0.000514,0.262402
6,0.000212,6.898602,0.414015,0.488332,0.000270,0.137618
7,0.000212,6.898602,0.414015,0.488332,0.000270,0.137618
8,-0.000338,7.152254,-0.719188,0.170908,0.000188,0.096120
9,-0.000338,7.152254,-0.719188,0.170908,0.000188,0.096120


In [154]:
dft = dft.join(linr_res)

In [155]:
dft =pd.concat([loc1,loc2, dft], axis =1)
dft.to_csv('SlopesStats.csv')